In [4]:
import pandas as pd 
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("../testing_ground/final_dataset1.csv")
df.head(10)
# df['name'].nunique()

,user_id,name,rating,appid
0,151603712,the elder scrolls v skyrim,5,72850
1,59945701,the elder scrolls v skyrim,3,72850
2,92107940,the elder scrolls v skyrim,5,72850
3,250006052,the elder scrolls v skyrim,5,72850
4,11373749,the elder scrolls v skyrim,5,72850
5,54103616,the elder scrolls v skyrim,2,72850
6,56038151,the elder scrolls v skyrim,1,72850
7,94088853,the elder scrolls v skyrim,5,72850
8,116085629,the elder scrolls v skyrim,4,72850
9,148510973,the elder scrolls v skyrim,4,72850


In [12]:
user_ratings = df.pivot_table(index=['user_id'],columns=['appid'],values='rating')
user_ratings = user_ratings.dropna(thresh=0,axis=1).fillna(0)
user_ratings

user_id,5250,76767,86540,229911,298950,381543,547685,554278,561758,577614,...,309107542,309181805,309188905,309213952,309216884,309255941,309265377,309404240,309554670,309626088
appid,,,,,,,,,,,,,,,,,,,,,
10,0.0,5.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
409720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
729040,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
similarity_ratings = user_ratings.T
indices = df['appid'].nunique()
corr_np = np.zeros(shape=(indices,indices))
indices = df['appid'].unique()

print(corr_np.shape)
corr_matrix = pd.DataFrame(index=indices,columns=indices,)
corr_matrix.fillna(0)
# corr_matrix['fallout 4']
count=0
i=0
for index, row in similarity_ratings.iterrows():
    print(i)
    j=0
    for index2,row2 in similarity_ratings.iterrows():
        # print(i,j)
        # value = row.corr(row2,method='pearson')
        np_row1 = row.to_numpy(copy=True)
        np_row2 = row2.to_numpy(copy=True)
        x_bar = np_row1.mean()
        # print(x_bar)
        y_bar = np_row2.mean()
        x_minus_x_bar = np_row1 - x_bar
        y_minus_y_bar = np_row2 - y_bar
        s_xy = sum(np.multiply(x_minus_x_bar,y_minus_y_bar))
        s_xx = sum(np.multiply(x_minus_x_bar,x_minus_x_bar))
        s_yy = sum(np.multiply(y_minus_y_bar,y_minus_y_bar))
        r = s_xy/np.sqrt(s_xx*s_yy)
        if corr_np[i,j] == 0:
            corr_np[i,j] = r
        if corr_np[j,i] == 0:
            corr_np[j,i] = r
        j+=1
    i+=1
        # corr_matrix
# print(count)

        # for i in range(len(np_row1)):
        #     x_minus_x_bar = 0
        #     y_minus_y_bar = 0
        #     x_minus_x_bar = np_row1[i] - x_bar
        #     s_xx += (x_minus_x_bar)**2
        #     y_minus_y_bar = np_row2[i] - y_bar
        #     s_yy = y_minus_y_bar**2
        #     s_xy += x_minus_x_bar*y_minus_y_bar

        # corr_matrix[index][index2] = value
print(corr_np)


(327, 327)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274

In [7]:
corr_matrix = pd.DataFrame(corr_np,index=indices,columns=indices)
corr_matrix.to_csv("../data/correlation_matrix")
corr_matrix[292030]

72850    -0.005778
377160   -0.006281
17390     0.000394
22380     0.008592
550      -0.003260
            ...   
313120    0.019940
383790   -0.004426
298630    0.010508
6000      0.030381
9450     -0.003980
Name: 292030, Length: 327, dtype: float64

In [13]:
## Building the similarity matrix
game_similarity_df = user_ratings.corr(method='pearson',min_periods=0)
game_similarity_df = corr_matrix
# game_similarity_df.to_csv("correlatedMatrix.csv",index=True)
# game_similarity_df['fallout 4']['the ship']

appid,10,30,50,70,80,130,220,240,280,300,...,383790,386360,387290,389570,391540,409710,409720,729040,765570,911400
appid,,,,,,,,,,,,,,,,,,,,,
10,1.000000,0.105720,-0.000538,0.001608,0.326551,0.006108,0.020488,0.091387,0.019657,0.022356,...,0.016663,0.012480,0.020343,-0.010901,-0.010222,-0.009213,-0.007704,-0.001022,0.021931,0.006047
30,0.105720,1.000000,0.065879,0.054617,0.056708,0.085334,0.017123,0.030449,-0.002663,0.092928,...,0.011448,-0.003815,-0.002730,-0.003199,-0.003000,-0.004979,-0.003660,0.009181,-0.003119,-0.003897
50,-0.000538,0.065879,1.000000,0.444784,-0.000346,0.678435,0.185390,0.070423,0.083868,0.064495,...,0.011230,-0.003867,-0.002768,-0.003243,0.031115,0.063899,0.047164,0.052331,-0.003161,0.023676
70,0.001608,0.054617,0.444784,1.000000,0.006668,0.365009,0.239420,0.073498,0.041464,0.046361,...,0.058840,0.006626,-0.003704,-0.004340,0.030514,0.056073,0.068162,0.088093,-0.004231,-0.005286
80,0.326551,0.056708,-0.000346,0.006668,1.000000,-0.004961,0.014796,0.074357,0.013105,0.016126,...,0.020434,0.000103,0.002949,-0.005828,-0.005465,-0.009069,-0.006667,-0.002630,-0.005681,0.012958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409710,-0.009213,-0.004979,0.063899,0.056073,-0.009069,0.046749,0.136470,0.051614,0.045073,0.007152,...,0.049611,0.007553,0.086630,-0.005574,0.029446,1.000000,0.462843,0.175605,0.059246,0.144647
409720,-0.007704,-0.003660,0.047164,0.068162,-0.006667,0.070180,0.119875,0.031000,0.020409,0.022879,...,-0.003717,0.009146,0.084079,-0.004097,-0.003842,0.462843,1.000000,0.171659,0.025060,0.076304
729040,-0.001022,0.009181,0.052331,0.088093,-0.002630,0.064535,0.159173,0.092363,0.046008,0.053022,...,0.056138,0.001713,0.006328,0.002402,0.026254,0.175605,0.171659,1.000000,0.146973,0.070680


In [9]:
def get_similar_games(game_name,user_rating):
    similar_score = game_similarity_df[game_name]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score.head(10)

# print(get_similar_games("the witcher 3 wild hunt",5))

In [10]:
# user_test = [("age of empires ii hd edition",5),("the witcher 3 wild hunt",2),("counterstrike global offensive",1),("portal",5)]

similar_games = pd.DataFrame()

for game,rating in user_test:
    similar_games = similar_games.append(get_similar_games(game,rating),ignore_index=True)

similar_games = similar_games.sum().sort_values(ascending=False)
similar_games.head(10)

NameError: name 'user_test' is not defined